In [18]:
import pandas as pd
import numpy as np
import re
import time
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, GRU, Embedding
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Input
tf.__version__

lines = open('cornell/movie_lines.txt').read().split('\n')
len(lines)


lines[:20]
#lineid, #characterID , #movieID , #chracter name , #text of the utterance

conv_lines = open('cornell/movie_conversations.txt').read().split('\n')
conv_lines[:10]
#characterid 1, characterid 2 ,movie id, lineids in chronological order

id2line = {}
for line in lines:
    line_list = line.split(' +++$+++ ')
    if len(line_list) == 5:
        id2line[line_list[0]] = line_list[4]
    else :
        print line_list

print id2line['L1045']

convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ","")
    convs.append(_line.split(','))

print convs[:10]

input_x = []
output_y = []
for conv in convs:
    for i in range(len(conv) - 1):
        input_x.append(id2line[conv[i]])
        output_y.append(id2line[conv[i+1]])

#for i in range(1,10):
#    print(input_x[i])
#    print(output_y[i])
##change from here
inputen_x = []
outputfr_y = []
with open("fra.txt") as f:
    for line in f:
        values = line.split('\t')
        inputen_x.append(values[0].rstrip())
        outputfr_y.append(values[1].rstrip())


#for i in range(1,10):
#    print inputen_x[i]
#    print outputfr_y[i]

##to here

    
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text


input_x_clean = []
for i in range(len(inputen_x)):
    input_x_clean.append(clean_text(inputen_x[i]))

output_y_clean = []
for j in range(len(outputfr_y)):
    output_y_clean.append(clean_text(outputfr_y[j]))
    
for i in range(1,10):
    print(input_x_clean[i])
    print(output_y_clean[i])
print ("length of input_x_clean is")
print(len(input_x_clean))
print ("length of output_x_clean is")
print(len(output_y_clean))
#input_x_clean = [:10000]
#output_y_clean = [:10000]
min_length = 2
max_length = 20
final_input = []
final_output = []
for i in range(0,len(input_x_clean)):
    temp_input_list = input_x_clean[i].split(' ')
    temp_output_list = output_y_clean[i].split(' ')
    if len(temp_input_list) > max_length or len(temp_input_list) < min_length  or len(temp_output_list) > max_length or len(temp_output_list) < min_length:
        d  = 1 + 1
    else:
        final_input.append(input_x_clean[i])
        final_output.append(output_y_clean[i])
    
print ("length of final input is")
print (len(final_input))
print ("length of final output is")
print (len(final_output))



vocab_en = {}
for sent in final_input:
    for word in sent.split():
        if word not in vocab_en:
            vocab_en[word] = 1
        else:
            vocab_en[word] += 1

vocab_fr = {}
for sent in final_output:
    for word in sent.split():
        if word not in vocab_fr:
            vocab_fr[word] = 1
        else:
            vocab_fr[word] += 1

print("length of vovabulary is")
print len(vocab_en)
print len(vocab_fr)

word_num = 0
threshold = 30
codes  = ['<EOS>', '<UNK>', '<GO>']
vocab_to_int_output = {}
vocab_to_int_output['<PAD>'] = 0
for code in codes:
    vocab_to_int_output[code] = len(vocab_to_int_output)
for word, count in vocab_fr.items():
    if count >= threshold:
        vocab_to_int_output[word] = word_num+4
        word_num += 1

vocab_to_int_input = {}
for word, count in vocab_en.items():
    if count >= threshold:
        vocab_to_int_input[word] = len(vocab_to_int_input)
vocab_to_int_input['<UNK>'] = len(vocab_to_int_input)    
print ("length of vocab to int is")
print(len(vocab_to_int_output))
print (len(vocab_to_int_input))

int_to_vocab_input = {v_i : v for v, v_i in vocab_to_int_input.items()}
int_to_vocab_output = {v_i : v for v, v_i in vocab_to_int_output.items()}
print ("length of int to vocab is")
print(len(int_to_vocab_input))
print(len(int_to_vocab_output))



for i in range(len(final_output)):
    final_output[i] += ' <EOS>'
for i in range(len(final_output)):
    final_output[i] = '<GO> ' + final_output[i]
print (final_output[0])

input_int = []
for sent in final_input:
    temp_int = []
    for word in sent.split( ' ' ):
        if word in vocab_to_int_input:
            temp_int.append(vocab_to_int_input[word])
        else:
            temp_int.append(vocab_to_int_input['<UNK>'])
    input_int.append(temp_int)
output_int = []
for sent in final_output:
    temp_int = []
    for word in sent.split( ' ' ):
        if word in vocab_to_int_output:
            temp_int.append(vocab_to_int_output[word])
        else:
            temp_int.append(vocab_to_int_output['<UNK>'])
    output_int.append(temp_int)
    
print ("length of output in and input int")
print (len(input_int))
print (len(output_int))
print ("checking output int and input int")
print (input_int[1])
print (output_int[1])

max_length = 0
for i in range (len(input_int)):
    if len(input_int[i]) > max_length:
        max_length = len(input_int[i])

print ("maximum length of a line is for input")
print (max_length)

import math
min_length = float('inf')
for i in range (len(input_int)):
    if len(input_int[i]) < min_length:
        min_length = len(input_int[i])
        print (input_int[i])

print ("min length of a line is for input")
print (min_length)

sorted_input = []
sorted_output = []
for length in range(min_length, max_length+1):
    for i in enumerate(input_int):
        if len(i[1]) == length:
            sorted_input.append(input_int[i[0]])
            sorted_output.append(output_int[i[0]])
            
print ("length of sorted input is")
print (len(sorted_input))
print (len(sorted_output))
print ("length of sorted output is")

for i in range(4):
    print sorted_output[i]
    
## PAD is 0 , EOS is 1 , UNK is 2 and GO is 3

np_encoder_input = np.zeros([len(sorted_input), len(max(sorted_input, key = lambda x : len(x)))], dtype = np.int32)
for i,j in enumerate(sorted_input):
    np_encoder_input[i][0:len(j)] = j
np_decoder_input = np.zeros([len(sorted_output), len(max(sorted_output, key = lambda x : len(x)))], dtype = np.int32)
for i,j in enumerate(sorted_output):
    np_decoder_input[i][0:len(j)] = j
    
print ("shape of encoder and decoder input is")
print (np_encoder_input.shape)
print (np_decoder_input.shape)

['']
They do not!
[['L194', 'L195', 'L196', 'L197'], ['L198', 'L199'], ['L200', 'L201', 'L202', 'L203'], ['L204', 'L205', 'L206'], ['L207', 'L208'], ['L271', 'L272', 'L273', 'L274', 'L275'], ['L276', 'L277'], ['L280', 'L281'], ['L363', 'L364'], ['L365', 'L366']]
run
cours 
run
courez 
fire
au feu 
help
À l'aide 
jump
saute
stop
Ça suffit 
stop
stop 
stop
arrêtetoi 
wait
attends 
length of input_x_clean is
154883
length of output_x_clean is
154883
length of final input is
154239
length of final output is
154239
length of vovabulary is
14235
31348
length of vocab to int is
2460
1938
length of int to vocab is
1938
2460
<GO> je comprends <EOS>
length of output in and input int
154239
154239
checking output int and input int
[924, 809]
[3, 1164, 1596, 2, 1]
maximum length of a line is for input
20
[924, 332]
min length of a line is for input
2
length of sorted input is
154239
154239
length of sorted output is
[3, 1027, 2129, 1]
[3, 1164, 1596, 2, 1]
[3, 1027, 1835, 2, 2, 1]
[3, 2, 221, 2, 1

In [2]:
#encoder_input  = [num_of_sentences, length]  , decoder_input = [number_of_sentences, length] 
#the vocablury size is 7878 the sequence length for encoder is 20 while for decoder its 22.
#Now we will make embedding layer for incoder and decoder 
#Input shape

#2D tensor with shape: (batch_size, sequence_length).

#Output shape


#3D tensor with shape: (batch_size, sequence_length, output_dim).



In [1]:
import pandas as pd
import numpy as np
import re
import time
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, GRU, Embedding
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Input
tf.__version__

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'1.7.0'

In [14]:
import random
n_data = 100000
input_len = 5
output_len = 7
output_data = np.zeros((n_data,output_len),dtype=np.int32)
input_data = np.zeros((n_data, input_len), dtype = np.int32)
print output_data.shape
print input_data.shape
for i in range(n_data):
    for j in range(input_len):
        input_data[i][j] = random.randint(3,30)
        
for i in range(n_data):
    output_data[i][0] = 0
    output_data[i][6] = 30
    for j in range(1,6):
        output_data[i][j] = input_data[i][j-1]

        
print input_data[0]
print output_data[0]

(100000, 7)
(100000, 5)
[16  4 11 20 25]
[ 0 16  4 11 20 25 30]


In [15]:
vocab_size_input = 31
vocab_size_output = 31
encoder_seq_length = 5
decoder_seq_length = 7
latent_dim = 32
num_encoder_tokens = vocab_size_input
num_decoder_tokens = vocab_size_output
epochs = 10
batch_size = 128
total_data = 100000

In [4]:
print (np_encoder_input[1])
print (np_decoder_input[1])
import gc
del conv_lines
del lines
del convs
del input_x, output_y
del input_x_clean, output_y_clean
del final_input,final_output
del input_int, output_int
del sorted_input,sorted_output


NameError: name 'np_encoder_input' is not defined

In [16]:
np_encoder_input = input_data
np_decoder_input = output_data
decoder_input_data = np_decoder_input[:, :-1]
decoder_target_data = np_decoder_input[:, 1:]
encoder_input_data = np.copy(np_encoder_input)
print (decoder_input_data.shape)
print (encoder_input_data.shape)



(100000, 6)
(100000, 5)


In [17]:

print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)
max= 0 
for dataarray in decoder_target_data:
    for data in dataarray:
        if max < data:
            max=data

print max

(100000, 5)
(100000, 6)
(100000, 6)
30


In [20]:
import random
def generate_data(encoder_input_data,decoder_input_data,decoder_target_data):
    batch_size = 512
    re_encoder_input = np.zeros((512,5), dtype = np.int32)
    re_decoder_input = np.zeros((512,6), dtype= np.int32)
    re_decoder_target = np.zeros((512,6,31), dtype= np.int32)
    counter = 0
    max_i = len(encoder_input_data)
    while True:
        while counter < batch_size :
            i = random.randint(0,max_i)
            re_encoder_input[counter] = encoder_input_data[i][:]
            re_decoder_input[counter] = decoder_input_data[i][:]
            k = 0
            while k < len(decoder_target_data[i]):
                re_decoder_target[counter][k][decoder_target_data[i][k]] = 1
                k += 1
            counter += 1
        
        yield [re_encoder_input, re_decoder_input] , re_decoder_target

        



In [21]:
#latent_dim = 64
#num_encoder_tokens = 374
#num_decoder_tokens = 374
#epochs = 10
#batch_size = 64
encoder_inputs = Input(shape=(None,))
x = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
x, state_h, state_c = LSTM(512,
                           return_state=True)(x)
encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,))
decoder_embedding_x = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
decoder_lstm = LSTM(512, return_sequences=True, return_state=True)

x,_,_ = decoder_lstm(decoder_embedding_x, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(x)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit_generator(generate_data(encoder_input_data, decoder_input_data, decoder_target_data),
          steps_per_epoch = 194,epochs = 100
          
          )

Epoch 1/100
194/194 [==============================] - 143s 739ms/step - loss: 2.5211
Epoch 2/100
194/194 [==============================] - 144s 742ms/step - loss: 1.2695
Epoch 3/100
194/194 [==============================] - 145s 747ms/step - loss: 0.3531
Epoch 4/100
194/194 [==============================] - 145s 746ms/step - loss: 0.0664
Epoch 5/100
194/194 [==============================] - 158s 813ms/step - loss: 0.0430
Epoch 6/100
194/194 [==============================] - 162s 837ms/step - loss: 0.0099
Epoch 7/100
194/194 [==============================] - 147s 756ms/step - loss: 0.0117
Epoch 8/100
194/194 [==============================] - 143s 738ms/step - loss: 0.0105
Epoch 9/100
194/194 [==============================] - 143s 739ms/step - loss: 0.0062
Epoch 10/100
194/194 [==============================] - 145s 746ms/step - loss: 1.9135e-06
Epoch 11/100
194/194 [==============================] - 166s 854ms/step - loss: 5.4843e-07
Epoch 12/100
194/194 [======================

194/194 [==============================] - 162s 833ms/step - loss: 1.1921e-07
Epoch 92/100
194/194 [==============================] - 163s 840ms/step - loss: 1.1921e-07
Epoch 93/100
194/194 [==============================] - 161s 832ms/step - loss: 1.1921e-07
Epoch 94/100
194/194 [==============================] - 163s 841ms/step - loss: 1.1921e-07
Epoch 95/100
194/194 [==============================] - 165s 848ms/step - loss: 1.1921e-07
Epoch 96/100
194/194 [==============================] - 163s 840ms/step - loss: 1.1921e-07
Epoch 97/100
194/194 [==============================] - 167s 860ms/step - loss: 1.1921e-07
Epoch 98/100
194/194 [==============================] - 164s 846ms/step - loss: 1.1921e-07
Epoch 99/100
194/194 [==============================] - 162s 835ms/step - loss: 1.1921e-07
Epoch 100/100
194/194 [==============================] - 161s 832ms/step - loss: 1.1921e-07


In [22]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(512,))
decoder_state_input_c = Input(shape=(512,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding_x, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [23]:
encoder_model.summary()
decoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, None, 32)          992       
_________________________________________________________________
lstm_5 (LSTM)                [(None, 512), (None, 512) 1116160   
Total params: 1,117,152
Trainable params: 1,117,152
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________


In [24]:
def input_seq_to_num(sent):
    output_sent = np.zeros((1,20),dtype=np.int32)
    initial_list = []
    sent = clean_text(sent)
    for words in sent.split(' '):
        if words in vocab_to_int_input:
            initial_list.append(vocab_to_int_input[words])
        else:
            initial_list.append(vocab_to_int_input['<UNK>'])
            
    output_sent[0][0:len(initial_list)] = initial_list
    output_sent[0] = output_sent[0][::-1]
    return output_sent
    
#input_seq =  input_seq_to_num("hello how are you")
    

In [ ]:
#states_value = encoder_model.predict(input_seq)

In [ ]:
#print (states_value[0].shape)
#print (states_value[1].shape)

In [25]:
def speak(input_sent):
    states_value = encoder_model.predict(input_sent)
    encoder_input_seq =  np.zeros((1,1), dtype=np.int32)
    encoder_input_seq[0][0]  = 0
    stop_condition = False
    decoded_sent = []
    max_decoder_seq_length = 10
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([encoder_input_seq] + states_value)

        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        print sampled_token_index
        sampled_char = sampled_token_index
        decoded_sent.append(sampled_char)
        if (sampled_char == 30 or len(decoded_sent) > max_decoder_seq_length):
            stop_condition = True
        encoder_input_seq =  np.zeros((1,1), dtype=np.int32)
        encoder_input_seq[0][0]  = sampled_token_index
        states_value = [h, c]
    return 



In [26]:
print (speak([21,14 ,18 ,11, 11,  5, 29,  19 , 18, 26,  15, 23, 17, 12, 18, 14,  16, 13,  0,  0]))


14
21
3
21
5
30
None


In [13]:
print (speak([19 ,25  ,4  ,4  ,3 ,29 ,22  ,4 ,30  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0]))
print (speak([26 ,30 ,19 ,28 ,14 ,18 ,26  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0  ,0]))

26
20
22
20
30
None
30
None
